# Table of Contents
1. [Things to Do](#Things-to-Do)
1. [Introduction](#Introduction)
1. [Set up Python](#Set-up-Python)
2. [Functions](#Functions)
3. [Getting Data](#Get-Data)
4. [Analyze Data](#Analyze-Data):
    - [Population Size](#Population-Size) - noticeable changes
    - [Sex Distribution](#Sex-Distribution) - does not change
    - [Tail Condition Distribution](#Tail-Condition-Distribution)- noticeable change
        - [Severity of Autotomy](#Severity-of-Autotomy)- noticeable change
    - [Location](#Location)
    - [Morphometrics](#Morphometrics):
        - [SVL](#SVL)
        - [TL](#TL)
        - [RTL](#RTL)
        - [Mass](#Mass)- noticeable change (Check Males)
    - [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
    - [Captures](#Captures)
    - [Growth](#Growth)
        - [SVL Growth](#SVL-Growth)
        - [TL Growth](#TL-Growth)
        - [RTL Growth](#RTL-Growth)
        - [Mass Growth](#Mass-Growth)
    - [Correlations to Population](#Correlations-to-Population)
5. [Export Files](#Export-Files)

# Things to Do

Also search for "to be done"

## [Resume Here](#resume)

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to describe the community of the _Sceloporus jarrovii_ and _Sceloporus virgatus_ lizards in the Crystal Creek wash from 2000 until 2017.  The population demographic metrics we examine are: [population size](#Population-Size), [sex distribution](#Sex-Distribution), [tail condition distribution](#Tail-Condition-Distribution), [location](#Location), [morphometrics](#Morphometrics) -- [SVL](#SVL), [TL](#TL), [RTL](#RTL), [mass](#Mass) --,  [survival and rates and likelihood of recapture](#Survival-and-Rates-and-Likelihood-of-Recapture), and [growth](#Growth).

We will examine these metrics and interactions among them with particular interest in the impact of environmental factors from year to year.


##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#Table-of-Contents)

In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *
from prettyprint import *


import plotly
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [2]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/outputFiles'
                         ,'log':'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/'
                         ,'output':'S:/Chris/TailDemography/TailDemography/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles/Descriptve/'}
             ,'gandolf':{'source':'C:/Users/craga/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/'
                           ,'output':'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}}

### Choose Device

In [3]:
device = deviceDict['gandolf']
device

{'source': 'C:/Users/craga/Google Drive/TailDemography/outputFiles',
 'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/',
 'output': 'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}

# Source Data


### Logging

In [4]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution) #delete this we will use scipy stats describe instead
- [monthlit](#monthlit)
- [description](#description)
- [vocab_run](#vocab_run)

### distribution
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [5]:
foo = [0,1,2,'r']
distribution(foo)

In [6]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [7]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [8]:
np.isnan(np.nan)

True

In [9]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [10]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

## description
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

In [11]:
def description(x,variable,percentage=False):
    if percentage:
            res = x[variable].describe()
            res[['mean','std','min','25%','50%','75%','max']] = res[['mean','std','min','25%','50%','75%','max']]\
            .apply(lambda x:x*100) 
#Need to Add CI calculation to this function
#             meanCI = 'not calculated'
    else:
        res = x[variable].describe() 
    res['siqr'] = (res['75%']-res['25%'])/2
    res['meanCI'] = 'not calculated'
    return res

### vocab_run
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*vocab_run* takes a list, joins its the first the elements with a separator placing a different separator between
     the penultimate and final members of the list and returns the result as a string
     :param x: a list of strings to be concatenated
     :param connector_dict: a dictionary with keys describing the size of the list and values indicating the type of
     connectors separate the list elements.
    
Here are a few examples of how *vocab_run* works.

In [12]:
print("Could you bring some {} please?".format(vocab_run(['foo','bar','stuffkins'])))

Could you bring some foo, bar and stuffkins please?


In [13]:
print("You can either have {}.  You'll have to make a choice."\
      .format(vocab_run(['foo','bar','stuffkins'],connector_dict={1: None, 2: ' or ', 'run': ', '})))

You can either have foo, bar or stuffkins.  You'll have to make a choice.


[Back to Functions](#Functions)

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

## Get Data
[Top](#Table-of-Contents)

Here we can set the locations from which we get data and to which we export it.

In [14]:
os.chdir(device['source'])
mysourcefiles = glob.glob('cleaned CC data 2000-2017_*.csv')
mysourcefiles

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv',
 'cleaned CC data 2000-2017_2019-03-10 14hrs42min.csv',
 'cleaned CC data 2000-2017_2019-03-12 21hrs48min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs52min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs55min.csv',
 'cleaned CC data 2000-2017_2019-04-25 00hrs58min.csv',
 'cleaned CC data 2000-2017_2019-04-25 01hrs00min.csv',
 'cleaned CC data 2000-2017_2019-05-02 00hrs17min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs03min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs08min.csv',
 'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [15]:
pd.to_datetime(mysourcefiles[0].split("_")[1].split(".csv")[0].split(' ')[0])

Timestamp('2019-01-31 00:00:00')

Automate getting the latest file

In [16]:
[latestFile for latestFile in mysourcefiles if \
 max({pd.to_datetime(afile.split("_")[1].split(".csv")[0].split(' ')[0]) \
      for afile in mysourcefiles}) == pd.to_datetime(latestFile.split("_")\
                                                     [1].split(".csv")[0].split(' ')[0])]

['cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [17]:
min({afile.split(' ')[-1].replace('hrs','').replace('min.csv','') for afile in mysourcefiles})

'0017'

In [18]:
latest = mysourcefiles[-1]
latest

'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv'

The most recent file is the one we will use as _df_ in our descriptive analysis.

In [19]:
df=pd.read_csv(latest)
df.date=pd.to_datetime(df.date)
df['month']=df.date.dt.month.apply(monthlit)
df.head()

,species,toes_orig,sex,date,svl,tl,rtl,autotomized,mass,location,meters,newRecap,painted,sighting,paint.mark,vial,misc,rtl_orig,toes,toe_pattern,description,action,pattern_b,replacement,year,tl_svl,mass_svl,all_meters,initialCaptureDate,year_diff,smallest_svl,svl_diff,liznumber,sex_count,daysSinceCapture,capture,month
0,j,3-7-11-19,m,2002-07-14,63.0,92.0,0.0,False,10.0,halfway up to site,-200,N,NaN,NaN,b101t,toes in vial 58-02,NaN,0.0,3-7-11-19,NaN,NaN,NaN,NaN,NaN,2002,1.460317,0.158730,['-200'],2002-07-14,0,63.0,0.0,375,1,0,1,Jul
1,j,3-7-11-18,m,2002-07-14,66.0,92.0,0.0,False,10.8,left downstream 100m v 1 falls,-100,N,NaN,NaN,b102t,toes in vial 59-02,NaN,0.0,3-7-11-18,NaN,NaN,NaN,NaN,NaN,2002,1.393939,0.163636,['-100' '87'],2002-07-14,0,66.0,0.0,374,1,0,1,Jul
2,j,3-7-12-16,m,2002-07-14,68.0,103.0,0.0,False,10.3,90m v 1 falls,-90,N,NaN,NaN,b103t,toes in vial 60-02,NaN,0.0,3-7-12-16,NaN,NaN,NaN,NaN,NaN,2002,1.514706,0.151471,['-90'],2002-07-14,0,68.0,0.0,376,1,0,1,Jul
3,j,10,m,2002-07-14,85.0,118.0,0.0,False,19.5,sb - trail intersection v 1 falls,-20,R,toe loss may be natural,NaN,b104t,NaN,NaN,0.0,10,NaN,NaN,NaN,NaN,NaN,2002,1.388235,0.229412,['-20' '-12' '-35'],2002-07-14,0,85.0,0.0,830,2,0,1,Jul
4,v,10-16,f,2002-07-03,63.0,49.0,21.0,True,11.6,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,N,painted; gravid; <5 mites in pockets,NaN,w1a,toes in vial 34-02 (10-16),NaN,21.0,10-16,NaN,NaN,NaN,NaN,NaN,2002,0.777778,0.184127,['-15'],2002-07-03,0,63.0,0.0,1414,1,0,1,Jul


## Analyze Data
[Top](#Table-of-Contents)

We will first examine the range and distribution of number of variables in our data set:
- [Population Size](#Population-Size)
- [Species Distribution](#Species-Distribution)
- [Sex Distribution](#Sex-Distribution)
- [Tail Condition Distribution](#Tail-Condition-Distribution)
- [Severity of Autotomy](#Severity-of-Autotomy)
- [Location](#Location)
- [Morphometrics](#Morphometrics):
    - [SVL](#SVL)
    - [TL](#TL)
    - [RTL](#RTL)
    - [Mass](#Mass)
- [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
- [Captures](#Captures)
- [Growth](#Growth)
    - [SVL Growth](#SVL-Growth)
    - [TL Growth](#TL-Growth)
    - [RTL Growth](#RTL-Growth)
    - [NTL Growth](#NTL-Growth)
    - [Mass Growth](#Mass-Growth)
We will use first captures of each lizard in a year for these analyses.
- [Correlations to Population](#Correlations-to-Population)

## Reducing the analyses sample by date range and capture

In [20]:
monthsExcluded = ['Oct','Dec']
idx_exclusion = (df.month.isin(monthsExcluded))&(df.capture==1)&(df.species!='j')
print("The number of individuals captured for the first time in {} is {}. \
These are excluded for further analyses.".format(vocab_run(monthsExcluded,{1: None, 2: ' or ', 'run': ', '}),
                                                df.loc[idx_exclusion].liznumber.nunique()))
df=df.loc[~idx_exclusion]
df=df.loc[(df.species=='j')& (df.sex.isin(['m','f']))]

The number of individuals captured for the first time in Oct or Dec is 1. These are excluded for further analyses.


Here we create datasets including only the first or last sighting in each year for a given animal.

In [21]:
df_lastInYear = df.loc[~df.duplicated(subset=['liznumber','year'],keep='last')]
df_firstInYear = df.loc[~df.duplicated(subset=['liznumber','year'])]

### Population Size

We will begin by examining the range and distribution of individuals in the population.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Add a column of counts of lizards weighted by the person-hours for the year, or provide argument based on asymptote analysis that we captured all lizards in the site.

In [22]:
populationSize = df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()
populationSize['percChange'] = -(1-populationSize.liznumber/populationSize.liznumber.shift())
populationSize

,year,liznumber,percChange
0,2000,153,NaN
1,2001,135,-0.117647
2,2002,119,-0.118519
3,2003,97,-0.184874
4,2004,70,-0.278351
5,2005,79,0.128571
6,2006,66,-0.164557
7,2007,94,0.424242
8,2008,88,-0.063830
9,2009,105,0.193182


 For some reason the first trace dissappears after I add the second one.  Need to fix this

In [23]:
popSize = go.Scatter(x = populationSize.year
           , y = populationSize.liznumber
          ,name = 'population size')
percentChange = go.Scatter(x = populationSize.year
           , yaxis = 'y2', y = populationSize.percChange
          ,name = 'percent change')
data = [popSize]#,percentChange]
layout = go.Layout(
    title = 'Population Size for Sceloporus jarrovii 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero'
        ),
    yaxis2 = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero',
        side = 'right'
        )
)
fig = go.Figure(
        data = data,
        layout = layout)
plot(fig, filename = 'Population Size for Sceloporus jarrovii.html')
iplot(fig, filename = 'Population Size for Sceloporus jarrovii.html')
# pio.to_image(fig, format='html')

We see a large decline in the <i>S. jarrovii</i> population size.

In [24]:
description(populationSize,'liznumber')

count                 18
mean             81.7222
std              31.0309
min                   44
25%                 56.5
50%                   76
75%                96.25
max                  153
siqr              19.875
meanCI    not calculated
Name: liznumber, dtype: object

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Sex Distribution

We will begin by examining the range and distribution of _sex_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Maybe aymptote analysis should be extended to examine asymptote by demographic factors as well.

In [25]:
populationSize_sex = df_firstInYear.groupby(['year','sex']).liznumber.nunique().reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'}),
       on=['year'])
populationSize_sex\
.loc[populationSize_sex.sex=='m','propMale'] = populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumberYear
populationSize_sex\
.loc[populationSize_sex.sex=='f','propFemale'] = (populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumberYear)
populationSize_sex.to_csv(device['output']+'population size.csv',index = False)
populationSize_sex

,year,sex,liznumber,liznumberYear,propMale,propFemale
0,2000,f,84,153,NaN,0.549020
1,2000,m,69,153,0.450980,NaN
2,2001,f,72,135,NaN,0.533333
3,2001,m,63,135,0.466667,NaN
4,2002,f,67,119,NaN,0.563025
5,2002,m,52,119,0.436975,NaN
6,2003,f,54,97,NaN,0.556701
7,2003,m,43,97,0.443299,NaN
8,2004,f,38,70,NaN,0.542857
9,2004,m,32,70,0.457143,NaN


In [26]:
Sjarrovii = go.Scatter(x = populationSize_sex.loc[(populationSize_sex.propMale.notna())].year
           , y = populationSize_sex.loc[(populationSize_sex.propMale.notna())].propMale
          ,name = '<i>S. jarrovii</i>')
data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion of Males by Year for <i>Sceloporus jarrovii</i> 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat = ".2%",
        title = 'Percentage of Males',
        titlefont = dict(
            size = 18),
    range = [0,1])
    
)
fig = go.Figure(
        data = data,
        layout = layout)
plot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii 2000-2017.html')
iplot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii 2000-2017.html')

In [27]:
description(populationSize_sex,'propMale',True)[['mean','std','min','25%','50%','75%','max','siqr']]

mean    45.5842
std     4.61074
min     34.0909
25%      43.732
50%     45.4545
75%     49.0281
max     54.4304
siqr    2.64804
Name: propMale, dtype: object

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

To be Done

There is not much variation in the sex ratio in the population here, suggesting that these drops do not differentially  impact males or females overall.  We should confirm that there isn't a difference in particular age/size groups.  Such a difference might be explained by developmental or social changes which leave certain groups more vulnerable than others.

### Tail Condition Distribution

Here we look at the proportion of individuals in our data who have experienced autotomy.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [28]:
populationSize_aut = df_firstInYear.groupby(['year','autotomized']).liznumber.nunique()\
.reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'})
       ,on=['year'])
populationSize_aut\
.loc[populationSize_aut.autotomized,'propAutotomized'] = populationSize_aut\
.loc[populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[populationSize_aut.autotomized].liznumberYear
populationSize_aut\
.loc[~populationSize_aut.autotomized,'propIntact'] = (populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumberYear)
populationSize_aut

,year,autotomized,liznumber,liznumberYear,propAutotomized,propIntact
0,2000,False,106,153,NaN,0.692810
1,2000,True,47,153,0.307190,NaN
2,2001,False,97,135,NaN,0.718519
3,2001,True,38,135,0.281481,NaN
4,2002,False,86,119,NaN,0.722689
5,2002,True,33,119,0.277311,NaN
6,2003,False,72,97,NaN,0.742268
7,2003,True,25,97,0.257732,NaN
8,2004,False,45,70,NaN,0.642857
9,2004,True,25,70,0.357143,NaN


In [29]:
Sjarrovii = go.Scatter(x = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].year
           , y = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].propAutotomized
          ,name = 'Autotomized <i>S. jarrovii</i>')



data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion Autotomized by Year for <i>Sceloporus jarrovii</i> 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat=".2%",
        title = 'Proportion of Autotomized Lizards',
        titlefont = dict(
            size = 18),
    range=[0,1])
)
fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii 2000-2017.html')
plot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii 2000-2017.html')

'Proportion Autotomized by Year for Sceloporus jarrovii 2000-2017.html'

In [30]:
description(populationSize_aut, 'propAutotomized',True)[['mean','std','min','25%','50%','75%','max','siqr']]

mean    30.4747
std     6.34579
min     14.8936
25%     27.6471
50%     29.9242
75%     36.2013
max          40
siqr    4.27712
Name: propAutotomized, dtype: object

Proportion of _S. jarrovii_ with evidence of autotomy is also oscillates over the years.  
To be done:
- does this track with oscillations in the population?

### Severity of Autotomy
Here we will regress SVL and sex on TL for intact _S. jarrovii_ to estimate what the TL would have been had an animal not autotomized.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [31]:
dfPred = df_firstInYear[['liznumber','svl','sex','tl','autotomized','year']]

In [32]:
dfPred['tl_svl'] = dfPred.tl/dfPred.svl

In [33]:
intactRatio = go.Box(y=dfPred.loc[dfPred.autotomized==False].tl_svl, name = 'Intact')
autotomizedRatio = go.Box(y=dfPred.loc[dfPred.autotomized].tl_svl, name = 'Autotmized')
data = [autotomizedRatio, intactRatio]
layout = go.Layout(
    title = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Autotomy Status',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL/SVL Ratio',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)

iplot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017.html')
# plot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017.html')

We will identify the _Sceloporus jarrovii_ with a tl/svl ratio outside of the center 95%.  To do this, we create a boolean column _extremeRatio_ individuals with tl/svl ratios outside of the center 95% have a value of **True** in this column.  We only use individuals without extreme ratios to model tl and svl.

In [34]:
print(dfPred.loc[dfPred.autotomized==False].tl_svl.describe())
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl>=dfPred.tl_svl.quantile(.025))&
                   (dfPred.tl_svl<=dfPred.tl_svl.quantile(.925)),'extremeRatio']= False
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl<dfPred.tl_svl.quantile(.025))|
                   (dfPred.tl_svl>dfPred.tl_svl.quantile(.925)),'extremeRatio']= True
dfPred.loc[dfPred.extremeRatio.isna()].autotomized.value_counts(dropna=False)


count    1036.000000
mean        1.354581
std         0.219160
min         0.140845
25%         1.315068
50%         1.354006
75%         1.397260
max         7.615385
Name: tl_svl, dtype: float64


True    434
Name: autotomized, dtype: int64

In [35]:
sjM = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].tl,
                     remove_na=True)
sjM

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-3.114711,0.659726,-4.721216,0.000003,0.980639,0.98059,-4.411695,-1.817727
1,svl,1.397366,0.009842,141.981905,0.000000,0.980639,0.98059,1.378018,1.416715


In [36]:
sjF = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].tl,
                     remove_na=True)
sjF

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,0.987045,0.691960,1.426447,0.154339,0.962916,0.962844,-0.372336,2.346426
1,svl,1.318262,0.011345,116.198414,0.000000,0.962916,0.962844,1.295975,1.340550


SVL and sex are sufficient to predict TL in males.

In [37]:
femaleIntercept = [thing for thing in sjF.loc[sjF.names=='Intercept','coef']][0]
maleIntercept = [thing for thing in sjM.loc[sjM.names=='Intercept','coef']][0]
femaleSvl = [thing for thing in sjF.loc[sjF.names=='svl','coef']][0]
maleSvl = [thing for thing in sjM.loc[sjM.names=='svl','coef']][0]

print(femaleIntercept,maleIntercept,femaleSvl,maleSvl)

0.9870448373167678 -3.1147108447449434 1.3182623599314436 1.3973660717218863


In [38]:
dfPred.loc[dfPred.sex=='f','tlPred'] = dfPred.loc[dfPred.sex=='f'].svl*femaleSvl+femaleIntercept
dfPred.loc[dfPred.sex=='m','tlPred'] = dfPred.loc[dfPred.sex=='m'].svl*maleSvl+maleIntercept
dfPred['diff'] = dfPred.tlPred - dfPred.tl
dfPred['propDiff'] = dfPred['diff']/dfPred.tlPred

In [39]:
dfPred.groupby(['autotomized','sex']).diff.describe()

count       mean        std        min        25%        50%  \
autotomized sex                                                                 
False       f    574.0  -0.692467   6.529064 -80.875544  -3.327674  -0.758968   
            m    462.0  -0.671692   7.219152 -19.014800  -3.679990  -0.862211   
True        f    221.0  26.518014  16.253555  -4.736362  15.220197  23.082786   
            m    214.0  29.482640  18.930467  -1.325425  16.967376  25.899597   

                       75%        max  
autotomized sex                        
False       f     1.400606  84.583672  
            m     1.594419  84.085111  
True        f    34.037574  86.174984  
            m    35.562064  96.648236

In [40]:
male = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male',mode='markers',marker=dict(size=5,opacity=0.8))
female = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female',mode='markers',marker=dict(size=5,opacity=0.8))
All = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['tl'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
maleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
femaleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
AllOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['tl'],
                    name = 'All Extreme',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))

data = [All,male,female,AllOut,maleOut,femaleOut]
layout = go.Layout(
    title = 'Scatter Plot of SVL vs TL for Intact <i>Sceloporus jarrovii</i> CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        tickangle=45,
        dtick = 1,
        title = 'SVL (mm)',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
#         tickformat = ".0%",
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfPred.loc[(dfPred.autotomized==False)]['count'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii CC 2000-2017.html')
plot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii CC 2000-2017.html')

'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii CC 2000-2017.html'

In [41]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['autotomized','sex']).propDiff.describe()

count      mean       std       min       25%       50%  \
autotomized sex                                                            
False       f    522.0  0.001014  0.048647 -0.073889 -0.033062 -0.005222   
            m    400.0  0.001032  0.042592 -0.079956 -0.031418 -0.003902   
True        f    221.0  0.295755  0.177319 -0.073702  0.168188  0.271755   
            m    214.0  0.294882  0.182797 -0.012196  0.178945  0.256071   

                      75%       max  
autotomized sex                      
False       f    0.023528  0.400620  
            m    0.021592  0.226929  
True        f    0.372975  0.851742  
            m    0.396617  0.921447

In [42]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index()

,year,autotomized,count,50%
0,2000,False,98.0,0.001034
1,2000,True,47.0,0.220697
2,2001,False,89.0,-0.006494
3,2001,True,38.0,0.267447
4,2002,False,74.0,-0.000452
5,2002,True,33.0,0.273869
6,2003,False,61.0,0.009045
7,2003,True,25.0,0.257109
8,2004,False,37.0,0.000447
9,2004,True,25.0,0.236155


In [43]:
dfYearlyEst = dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized','sex'])\
.propDiff.describe()[['count','50%']].reset_index(drop=False).merge(dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index().rename(columns = {'count':'count_all',
                                                                      '50%':'50%_all'}),
                                                                    how = 'left',
                                                                    on =['year','autotomized'])
dfYearlyEst.loc[dfYearlyEst.autotomized]

,year,autotomized,sex,count,50%,count_all,50%_all
2,2000,True,f,24.0,0.215635,47.0,0.220697
3,2000,True,m,23.0,0.224053,47.0,0.220697
6,2001,True,f,13.0,0.302205,38.0,0.267447
7,2001,True,m,25.0,0.263316,38.0,0.267447
10,2002,True,f,15.0,0.194670,33.0,0.273869
11,2002,True,m,18.0,0.297232,33.0,0.273869
14,2003,True,f,13.0,0.261006,25.0,0.257109
15,2003,True,m,12.0,0.229288,25.0,0.257109
18,2004,True,f,14.0,0.175265,25.0,0.236155
19,2004,True,m,11.0,0.257427,25.0,0.236155


In [44]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['count_all'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['50%_all'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017.html')
plot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017.html')

'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017.html'

In [45]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'],
                    name = 'All',mode='lines',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['min'],
                          dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['max']],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')
plot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')

'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html'

To be done:

- for each sex and all, regress prop tl on relevant count 
- add these lines to plots with CI
- report on stat (what does this tell us?)

# resume
[TOC](#Table-of-Contents)

In [46]:
male = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['count'].sum()))
female = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                    name = 'Female (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['count'].sum()))
All = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%'],
                    name = 'All (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count'].sum()))
data = [All,male,female,]
layout = go.Layout(
    title = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017.html')
plot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017.html')

'Boxplot Median Proportion of Tail Loss for CC 2000-2017.html'

In [47]:
dfPred.loc[(dfPred.autotomized==True)].groupby('sex').propDiff.describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
f,221.0,0.295755,0.177319,-0.073702,0.168188,0.271755,0.372975,0.851742
m,214.0,0.294882,0.182797,-0.012196,0.178945,0.256071,0.396617,0.921447


In [48]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')

In [49]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Morphometrics

In this section we describe the distributions of various morphometrics.

- [SVL](#SVL)
- [TL](#TL)
- [RTL](#RTL)
- [Mass](#SVL)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

#### SVL

Now we examine the range and distribution of svl values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of SVL](#SVLhist)

In [50]:
print("svl values in the data set range from {} to {} for and are distributed across sex \
as displayed here:"\
      .format(df_firstInYear.svl.min(), df_firstInYear.svl.max()))
description(df_firstInYear.groupby('sex'),variable='svl')

svl values in the data set range from 13.0 to 98.0 for and are distributed across sex as displayed here:


,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
sex,,,,,,,,,,
f,795.0,61.542138,14.155369,13.0,54.0,65.0,73.0,86.0,9.5,not calculated
m,676.0,68.008876,18.280204,24.0,55.0,73.0,83.0,98.0,14.0,not calculated


In [51]:
SVLbyYear = description(df_firstInYear.groupby(['year','sex']),'svl').reset_index()
SVLbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,84.0,60.071429,12.676499,31.0,54.00,60.0,68.50,83.0,7.250,not calculated
1,2000,m,69.0,68.289855,17.052774,31.0,60.00,70.0,83.00,98.0,11.500,not calculated
2,2001,f,72.0,58.736111,13.863513,37.0,43.00,58.5,71.00,82.0,14.000,not calculated
3,2001,m,63.0,65.650794,17.982212,36.0,49.00,67.0,81.50,93.0,16.250,not calculated
4,2002,f,67.0,60.417910,12.344977,27.0,55.00,60.0,70.00,80.0,7.500,not calculated
5,2002,m,52.0,69.346154,17.347526,27.0,63.00,69.5,83.25,92.0,10.125,not calculated
6,2003,f,54.0,55.537037,17.084378,26.0,36.25,62.0,69.00,80.0,16.375,not calculated
7,2003,m,43.0,61.348837,21.156879,29.0,35.00,65.0,80.50,89.0,22.750,not calculated
8,2004,f,38.0,60.631579,16.066452,30.0,58.25,66.0,72.25,80.0,7.000,not calculated
9,2004,m,32.0,67.500000,19.660013,29.0,64.50,72.0,80.00,91.0,7.750,not calculated


Let's plot these values. 

##### Figures of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [52]:
female = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')].svl
                      ,name='Female Box')
male = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')].svl
                    ,name='Males Box')
female_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['50%']
                      ,name='Female Median', line = dict(color = 'red'))
male_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['50%']
                    ,name='Male Median', line = dict(color = 'blue'))
female_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [male,female,male_line,female_line,male_count,female_count]
layout = go.Layout(
    title = 'Box Plot of SVL at Capture for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median SVL (mm)',
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii in CC 2000-2017.html')

'Box Plot of Median SVL Sceloporus jarrovii in CC 2000-2017.html'

To be done:
- look at correlation between the count and the median svl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

#### TL

Now we examine the range and distribution of TL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of TL](#TLhist)

In [53]:
TLbyYear = description(df_firstInYear.groupby(['year','autotomized','sex']),'tl').reset_index()
TLbyYear

,year,autotomized,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,False,f,60.0,77.233333,17.040696,38.0,71.75,78.5,88.25,114.0,8.250,not calculated
1,2000,False,m,46.0,87.413043,25.005400,38.0,80.00,90.0,108.50,122.0,14.250,not calculated
2,2000,True,f,24.0,67.208333,15.348075,30.0,61.00,66.5,77.25,93.0,8.125,not calculated
3,2000,True,m,23.0,76.347826,19.508993,30.0,64.00,75.0,93.00,108.0,14.500,not calculated
4,2001,False,f,59.0,77.627119,18.380929,50.0,57.50,79.0,92.50,109.0,17.500,not calculated
5,2001,False,m,38.0,80.236842,24.043243,48.0,55.00,79.0,100.00,124.0,22.500,not calculated
6,2001,True,f,13.0,62.846154,17.742821,35.0,56.00,66.0,71.00,92.0,7.500,not calculated
7,2001,True,m,25.0,71.960000,20.235036,23.0,66.00,80.0,83.00,103.0,8.500,not calculated
8,2002,False,f,52.0,79.942308,17.875839,31.0,73.75,81.0,92.50,106.0,9.375,not calculated
9,2002,False,m,34.0,91.117647,26.114857,34.0,80.00,92.0,110.75,130.0,15.375,not calculated


##### Histogram of TL (Intact)

In [54]:
femaleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&~(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&~(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Intact Females')
maleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Intact Males')
female_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxIntact,femaleBoxIntact,maleIntact,femaleIntact,male_count,female_count]
layout = go.Layout(
    title = 'TL for Intact <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL for Intact Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'TL for Intact Sceloporus jarrovii in CC 2000-2017.html')

'TL for Intact Sceloporus jarrovii in CC 2000-2017.html'

To be done:
- look at correlation between the count and the median tl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

##### Histogram of TL (Autotomized)

In [55]:
femaleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Autotomized Females')
maleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Autotomized Males')
female_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxAutotomized,femaleBoxAutotomized,maleAutotomized,femaleAutotomized,male_count,female_count]
layout = go.Layout(
    title = 'TL for Autotomized <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL for Autotomized Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'TL for Autotomized Sceloporus jarrovii in CC 2000-2017.html')

'TL for Autotomized Sceloporus jarrovii in CC 2000-2017.html'

##### Histogram of TL (Intact vs. Autotomized) 2

In [56]:
TLbyYear.groupby('year')['year','50%'].apply(lambda x: x[['50%']])

,50%
0,78.5
1,90.0
2,66.5
3,75.0
4,79.0
5,79.0
6,66.0
7,80.0
8,81.0
9,92.0


In [57]:
# BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
#                 y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
#                       ,name='Autotomized')
# BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
#               y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
#                     ,name='Intact')
compared = (TLbyYear.loc[~(TLbyYear.autotomized)]['50%'].reset_index(drop=True))\
-(TLbyYear.loc[(TLbyYear.autotomized)]['50%'].reset_index(drop=True))
Comparison = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
                   y = compared,name='Intact - Autotomized',
                       mode = 'markers')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [Comparison]
layout = go.Layout(
    title = 'Comparison on Median TL of <i>Sceloporus jarrovii</i> in CC 2000-2017 by Tail Condition',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii in CC 2000-2017.html')

'Comparison on Median TL of Sceloporus jarrovii in CC 2000-2017.html'

##### Histogram of TL (Intact vs. Autotomized)

In [58]:
BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
                      ,name='Autotomized')
BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
                    ,name='Intact')
# Autotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
#                    y = TLbyYear.loc[(TLbyYear.autotomized)]['50%']
#                       ,name='Autotomized')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [BoxAut,BoxIntact]
layout = go.Layout(
    title = 'TL of <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL of Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'TL of Sceloporus jarrovii in CC 2000-2017.html')

'TL of Sceloporus jarrovii in CC 2000-2017.html'

#### RTL

Now we examine the range and distribution of RTL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of RTL](#RTLhist)

In [59]:
RTLbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'rtl').reset_index()
RTLbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,24.0,22.083333,14.518105,0.0,8.25,25.0,32.25,48.0,12.000,not calculated
1,2000,m,23.0,21.826087,17.366771,0.0,3.00,24.0,33.50,55.0,15.250,not calculated
2,2001,f,13.0,17.923077,20.229770,0.0,0.00,7.0,32.00,50.0,16.000,not calculated
3,2001,m,25.0,18.760000,17.469688,0.0,2.00,16.0,29.00,58.0,13.500,not calculated
4,2002,f,15.0,13.600000,16.348001,0.0,0.50,5.0,26.50,44.0,13.000,not calculated
5,2002,m,18.0,26.222222,18.763405,0.0,6.25,30.0,41.00,54.0,17.375,not calculated
6,2003,f,13.0,15.461538,14.569462,1.0,4.00,9.0,32.00,39.0,14.000,not calculated
7,2003,m,12.0,20.750000,17.503896,0.0,7.00,18.0,26.50,56.0,9.750,not calculated
8,2004,f,14.0,12.714286,11.631711,0.0,4.75,10.5,15.00,36.0,5.125,not calculated
9,2004,m,11.0,18.454545,18.949214,0.0,0.00,19.0,31.00,52.0,15.500,not calculated


##### Histogram of rtl

In [60]:
femaleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='f')]['year'], 
                   y = RTLbyYear.loc[(RTLbyYear.sex=='f')]['50%']
                      ,name='Females')
maleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='m')]['year'], 
              y = RTLbyYear.loc[(RTLbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [maleAut,femaleAut]
layout = go.Layout(
    title = 'Median RTL for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median RTL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median RTL Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Median RTL Sceloporus jarrovii in CC 2000-2017.html')

'Median RTL Sceloporus jarrovii in CC 2000-2017.html'

#### Mass

Now we examine the range and distribution of mass values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Plot of mass](#Plot-of-mass)

We probably need to adjust this analysis to consider teh month in which the females were captures since gravidity or nearness to the time of parturition may bias the analysis.

In [61]:
MassbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'mass').reset_index()
MassbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,24.0,9.795833,3.652573,1.0,8.100,10.00,11.300,17.0,1.6000,not calculated
1,2000,m,23.0,15.043478,7.187858,2.0,9.300,16.00,18.450,31.0,4.5750,not calculated
2,2001,f,13.0,9.392308,4.353822,2.5,6.200,10.50,11.500,17.7,2.6500,not calculated
3,2001,m,25.0,14.644000,6.787493,1.0,10.600,13.70,20.700,26.3,5.0500,not calculated
4,2002,f,15.0,7.946667,3.147304,1.2,6.150,7.50,10.150,13.2,2.0000,not calculated
5,2002,m,18.0,13.605556,6.176947,1.2,9.275,14.55,18.125,23.4,4.4250,not calculated
6,2003,f,6.0,10.200000,4.874013,1.5,9.700,10.50,12.575,16.0,1.4375,not calculated
7,2003,m,10.0,11.540000,7.419973,1.0,7.475,11.80,14.950,24.2,3.7375,not calculated
8,2004,f,14.0,10.428571,4.892605,0.6,9.050,10.75,14.225,16.5,2.5875,not calculated
9,2004,m,10.0,13.200000,9.423847,0.5,3.875,16.00,21.000,23.8,8.5625,not calculated


##### Plot of mass

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [62]:
female = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='f')]['year'], 
                   y = MassbyYear.loc[(MassbyYear.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='m')]['year'], 
              y = MassbyYear.loc[(MassbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [male,female]
layout = go.Layout(
    title = 'Median Mass for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Mass (mm)',
        titlefont = dict(
            size = 18),
    range = [0,MassbyYear['50%'].max()+5]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Mass Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Median Mass Sceloporus jarrovii in CC 2000-2017.html')

'Median Mass Sceloporus jarrovii in CC 2000-2017.html'

## Captures

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.  We will use all captures for this

In [63]:
df_firstInYear.groupby('liznumber').capture.apply(lambda x: x.values)

liznumber
1                [1, 2]
2             [2, 1, 3]
3             [1, 2, 3]
4       [4, 1, 7, 5, 8]
5                [1, 3]
6                [1, 2]
7                [1, 2]
8                [1, 5]
9                [1, 3]
10            [1, 3, 2]
11            [3, 1, 5]
12               [1, 2]
13               [1, 2]
14               [1, 3]
15               [1, 2]
16            [1, 3, 2]
17               [1, 2]
18               [1, 2]
19                  [1]
20                  [1]
21                  [1]
22                  [1]
23                  [1]
24                  [1]
25                  [1]
26                  [1]
27                  [1]
28                  [1]
29            [5, 1, 6]
30                  [1]
31                  [1]
32                  [1]
33                  [1]
34                  [1]
35                  [1]
36            [3, 2, 1]
37                  [1]
38                  [1]
39                  [1]
40                  [1]
41                  [1]
42    

In [64]:
print("The maximum number of captures among lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').capture.max().min(),
              df_firstInYear.groupby('liznumber').capture.max().max()))
captureMedYr = description(df_firstInYear.groupby(['year','sex']),variable='capture')[['50%','75%','max']].reset_index()
captureMedYr

The maximum number of captures among lizards in the data set range from 1 to 18 for and are distributed across species and sex as displayed here:


,year,sex,50%,75%,max
0,2000,f,1.0,1.00,1.0
1,2000,m,1.0,1.00,1.0
2,2001,f,1.0,2.00,4.0
3,2001,m,1.0,1.00,3.0
4,2002,f,2.0,3.00,5.0
5,2002,m,1.5,3.00,5.0
6,2003,f,1.0,2.00,6.0
7,2003,m,1.0,2.50,6.0
8,2004,f,1.0,1.00,2.0
9,2004,m,1.0,1.00,3.0


In [65]:
female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
                   y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
              y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
                    ,name='Males')
# femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
#                    y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
#                       ,name='Females')
# maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
#               y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
#                     ,name='Males')

data = [male,female]#,maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18),
    range = [0,captureMedYr['50%'].max()+1]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Years Captured Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Median Years Captured  Sceloporus jarrovii in CC 2000-2017.html')

'Median Years Captured  Sceloporus jarrovii in CC 2000-2017.html'

In [66]:
# female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
#                    y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
#                       ,name='Females')
# male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
#               y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
#                     ,name='Males')
femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
                   y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
                      ,name='Females')
maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
              y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
                    ,name='Males')

data = [maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Years Captured Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Box Years Captured  Sceloporus jarrovii in CC 2000-2017.html')

'Box Years Captured  Sceloporus jarrovii in CC 2000-2017.html'

In order to interpret these data we need to factor in person-hours for each year.

## Years

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of years over which that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of years over which each lizard was captures and finally count the number of lizards that have a given number of years over which they were captured.  We will use all captures for this.

In [67]:
(df_firstInYear.groupby('liznumber').year_diff.max()+1).value_counts(normalize=True).reset_index()\
.rename(columns={'index':'year_diff','year_diff':'proportion'}).sort_values('year_diff')

,year_diff,proportion
0,1,0.786280
1,2,0.109938
2,3,0.050132
3,4,0.020229
4,5,0.008795
8,6,0.003518
7,7,0.003518
10,8,0.001759
14,9,0.000880
13,11,0.000880


In [68]:
print("The number of years over which each lizard was captured among lizards in the data set range from {} to {}\
for and are distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').year_diff.max().min()+1, 
              df_firstInYear.groupby('liznumber').year_diff.max().max()+1))
description(df_firstInYear.groupby('sex'),variable='year_diff')

The number of years over which each lizard was captured among lizards in the data set range from 1 to 17for and are distributed across species and sex as displayed here:


,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
sex,,,,,,,,,,
f,795.0,0.627673,2.015616,0.0,0.0,0.0,0.0,16.0,0.0,not calculated
m,676.0,0.460059,1.554883,0.0,0.0,0.0,0.0,15.0,0.0,not calculated


In [73]:
Males = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='m')].groupby('liznumber').capture.max(),
                     name='Males')
Females = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='f')].groupby('liznumber').capture.max(),
                       name='Females')

data = [Males,Females]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017.html')
plot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017.html')

'Histogram of Maximum Number of Captures by Sex for CC 2000-2017.html'

### Maximum Number of Captures based on Tail condition
 - match for species, sex, size and location

In [75]:
intactFemale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                   (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                      ,name='intact females')
intactMale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                 (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                      ,name='intact males')
autotomizedFemale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                        (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                           ,name='autotomized females')
autotomizedMale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                      (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                           ,name='autotomized males')

data = [intactFemale,intactMale,autotomizedFemale,autotomizedMale]
layout = go.Layout(
    title = 'Maximum Number of Captures by Tail Condition 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures by Tail Condition in Crystal Creek 2000 - 2017')

## Growth

- [SVL Growth](#SVL-Growth)
- [TL Growth](#TL-Growth)
- [RTL Growth](#RTL-Growth)
- [Mass Growth](#Mass-Growth)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

### SVL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate?

turn this into a function and then groupby and apply

df['svl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['svl'].max()-x['svl'].min())/(x['year_diff'].max()+1))

### TL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the tail size growth rate?

df['tl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['tl'].max()-x['tl'].min())/(x['year_diff'].max()+1))

### RTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the regrown tail size growth rate?

df['rtl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['rtl'].max()-x['rtl'].min())/(x['year_diff'].max()+1))

### NTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Mass Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate in terms of MAss?

df['mass_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['mass'].max()-x['mass'].min())/(x['year_diff'].max()+1))

tmp = df.groupby('liznumber').mass_growth_ann.nunique().reset_index()
check = tmp.loc[tmp.mass_growth_ann>1,'liznumber']
print(len(check))
check

## Correlations to Population

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [127]:
def candidate(m,dv,placement=(1,1)):
    assert(dv in m.columns)
    return m[dv].sort_values().reset_index().iloc[placement[0]:placement[1]+1,:]

from functools import reduce

def topcorr(corrdf,lowestrank,dvs):
    candidates = [candidate(corrdf,dv,(1,lowestrank)) for dv in dvs]
    merger =  reduce(lambda x, y: pd.merge(x, y, on = 'index', how = 'outer'), candidates).fillna('--')
    return merger

I need to create a df with with values to be correlated:
- count
- prop Male
- SVL
- TL
- RTL
- Mass
- TL Severity
- Sex Distr


In [138]:
dfCorrPrep = populationSize_sex.loc[populationSize_sex.sex=='f',
                                    ['year', 'sex', 'liznumber', 
                                     'liznumberYear','propFemale']]\
.merge(populationSize_aut.loc[populationSize_aut.propAutotomized.notna(),
                              ['year','liznumberYear', 'propAutotomized']], 
                         how = 'left', on = ['year','liznumberYear'])\
.merge(dfYearlyEst.loc[dfYearlyEst.autotomized,['year', 'sex', 
                                                'count_all', '50%_all']],
       on = ['year','sex'])\
.rename(columns = {'count_all':'nAutotomized', 
                   '50%_all':'medianPropTailAutotomized',
                                             'liznumber':'numberFemale'})
dfCorrPrep = dfCorrPrep[['liznumberYear', 
                         'propFemale', 'propAutotomized', 
                         'medianPropTailAutotomized']]
dfCorrPrep

,liznumberYear,propFemale,propAutotomized,medianPropTailAutotomized
0,153,0.549020,0.307190,0.220697
1,135,0.533333,0.281481,0.267447
2,119,0.563025,0.277311,0.273869
3,97,0.556701,0.257732,0.257109
4,70,0.542857,0.357143,0.236155
5,79,0.455696,0.367089,0.264371
6,66,0.545455,0.303030,0.284947
7,94,0.574468,0.148936,0.286744
8,88,0.500000,0.295455,0.328795
9,105,0.514286,0.276190,0.296802


In [139]:
#Dropping proportion of Females, but will put it back once I can order the y-axis
dfCorr = dfCorrPrep.corr()
dfCorr = topcorr(dfCorr,2,dfCorr.columns.tolist()) 
dfCorr = dfCorr.set_index('index')
dfCorr

,liznumberYear,propFemale,propAutotomized,medianPropTailAutotomized
index,,,,
propFemale,-0.224346,--,-0.291769,-0.0515601
medianPropTailAutotomized,-0.0432619,-0.0515601,-0.0771193,--
liznumberYear,--,-0.224346,--,-0.0432619


In [140]:
testx = dfCorr.columns
testy = dfCorr.index
testz = dfCorr.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
plot(test, filename = 'population correlation matrix.html')
iplot(test, filename = 'population correlation matrix.html')

## Export Files
[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)